## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-17-20-49-26 +0000,nypost,Trump says Ukraine might not need Tomahawks to...,https://nypost.com/2025/10/17/us-news/trump-sa...
1,2025-10-17-20-46-23 +0000,nyt,Live Updates: Trump Equivocates on Ukraine’s R...,https://www.nytimes.com/live/2025/10/17/us/tru...
2,2025-10-17-20-46-00 +0000,wsj,Opinion | Jensen Huang Is Wrong About China,https://www.wsj.com/opinion/jensen-huang-is-wr...
3,2025-10-17-20-37-30 +0000,nypost,Perv lured middle schooler into NYC park with ...,https://nypost.com/2025/10/17/us-news/perv-who...
4,2025-10-17-20-34-36 +0000,nyt,"Trump Threatens New Cuts to Federal Aid, Worke...",https://www.nytimes.com/2025/10/17/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2347/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,54
36,new,20
21,china,15
26,nyc,14
1,ukraine,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
355,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,128
321,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,110
311,2025-10-16-23-30-34 +0000,wapo,Top military officer central to Trump’s drug w...,https://www.washingtonpost.com/national-securi...,103
336,2025-10-16-21-54-57 +0000,cbc,"John Bolton, Trump's ex-national security advi...",https://www.cbc.ca/news/world/john-bolton-char...,102
272,2025-10-17-03-10-44 +0000,wapo,Trump refiles $15B suit against New York Times...,https://www.washingtonpost.com/business/2025/1...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
355,128,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
269,70,2025-10-17-03-46-08 +0000,nypost,"Zohran Mamdani dodges attacks, straight answer...",https://nypost.com/2025/10/16/us-news/mamdani-...
28,60,2025-10-17-19-56-00 +0000,wsj,Bank of Canada Gov. Tiff Macklem said data nex...,https://www.wsj.com/articles/bank-of-canada-su...
104,54,2025-10-17-16-42-19 +0000,nypost,John Bolton spotted still using White House ba...,https://nypost.com/2025/10/17/us-news/john-bol...
221,53,2025-10-17-10-00-00 +0000,wapo,"The average new-car price hit a record $50,000...",https://www.washingtonpost.com/business/2025/1...
0,45,2025-10-17-20-49-26 +0000,nypost,Trump says Ukraine might not need Tomahawks to...,https://nypost.com/2025/10/17/us-news/trump-sa...
45,43,2025-10-17-18-54-00 +0000,wsj,The number of unemployment claims filed by fed...,https://www.wsj.com/economy/jobs/unemployment-...
62,41,2025-10-17-18-19-43 +0000,nypost,Jeffrey Epstein-linked Prince Andrew forced to...,https://nypost.com/2025/10/17/world-news/princ...
248,40,2025-10-17-08-10-07 +0000,wsj,Stock Market Today: Dow Rises; Trump Says Extr...,https://www.wsj.com/livecoverage/stock-market-...
144,37,2025-10-17-15-18-45 +0000,nyt,"Baek Sehee, South Korean Author of ‘I Want to ...",https://www.nytimes.com/2025/10/17/arts/baek-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
